### Here we will explore an example of how to consolidate multiple csv files into a DataFrame and combined file


https://data.census.gov/cedsci/table?q=Population%20Total&tid=PEPPOP2019.PEPANNRES

In [2]:
import pandas as pd
import glob
import re

In [3]:
def get_year(path):
    pattern = re.compile('Y([0-9]{4})')
    year = re.search(pattern, path)
    if year:
        year = year[1]
    return year

path = "ACSDP1Y2011.DP05_data_with_overlays_2021-11-20T222122.csv"
print(get_year(path))

path = "ACSDP1Y2011.DP05_metadata_2021-11-20T222122.csv"
print(get_year(path))

2011
2011


In [4]:
def get_type(path):
    meta = re.search('metadata', path)
    if meta:
        return 'meta'
    else:
        return 'data'

path = "ACSDP1Y2011.DP05_data_with_overlays_2021-11-20T222122.csv"
print(get_type(path))

path = "ACSDP1Y2011.DP05_metadata_2021-11-20T222122.csv"
print(get_type(path))

data
meta


In [5]:
def get_year_type(path):
    return get_year(path),get_type(path)

path = "ACSDP1Y2011.DP05_data_with_overlays_2021-11-20T222122.csv"
print(get_year_type(path))

path = "ACSDP1Y2011.DP05_metadata_2021-11-20T222122.csv"
print(get_year_type(path))

('2011', 'data')
('2011', 'meta')


In [6]:
def fix_meta(df):
    df.columns = ['id', 'name']
    
    # expand the names into their own columns
    df = df[['id']].merge(df['name'].str.split('!!').apply(pd.Series), left_index = True, right_index = True)
    df.columns = ['id'] + ['level_{}'.format(x) for x in range(len(df.columns) - 1)]
    
    # keep only what I care about
    df = df.loc[df['level_0'].str.lower() == 'estimate']
    
    # set the corresponding index and drop id
    df.index = df['id']
    df = df.drop(columns = ['id'])
    
    return(df.reset_index())

In [7]:
def fix_data(df, year):
    # slice it based on the last observation
    df = df.iloc[-1]
    
    # create a multi-index level including the year. We need to create a list of tupples. 
    
    # option 1
    tuples = [(year, ind) for ind in df.index]
    
    # option 2
    tuples = list(zip(*[[year]*len(df), df.index]))
    index = pd.MultiIndex.from_tuples(tuples, names=["year", "name"])
    
    # assign the new index
    df.index = index
    
    # and just for clarity, let's name this
    df.name = 'data'
    
    return(df)

In [8]:
def fix_file(meta, *args):
    if meta == 'meta':
        return(fix_meta(*args))
    else:
        return(fix_data(*args))

In [9]:
path = '.\population_growth/*'
for file_path in glob.glob(path):
    year, meta = get_year_type(file_path)
    print(year,' | ', meta,' | ', file_path)

2010  |  data  |  .\population_growth\ACSDP1Y2010.DP05_data_with_overlays_2021-11-20T222122.csv
2010  |  meta  |  .\population_growth\ACSDP1Y2010.DP05_metadata_2021-11-20T222122.csv
2010  |  data  |  .\population_growth\ACSDP1Y2010.DP05_table_title_2021-11-20T222122.txt
2011  |  data  |  .\population_growth\ACSDP1Y2011.DP05_data_with_overlays_2021-11-20T222122.csv
2011  |  meta  |  .\population_growth\ACSDP1Y2011.DP05_metadata_2021-11-20T222122.csv
2011  |  data  |  .\population_growth\ACSDP1Y2011.DP05_table_title_2021-11-20T222122.txt
2012  |  data  |  .\population_growth\ACSDP1Y2012.DP05_data_with_overlays_2021-11-20T222122.csv
2012  |  meta  |  .\population_growth\ACSDP1Y2012.DP05_metadata_2021-11-20T222122.csv
2012  |  data  |  .\population_growth\ACSDP1Y2012.DP05_table_title_2021-11-20T222122.txt
2013  |  data  |  .\population_growth\ACSDP1Y2013.DP05_data_with_overlays_2021-11-20T222122.csv
2013  |  meta  |  .\population_growth\ACSDP1Y2013.DP05_metadata_2021-11-20T222122.csv
2013 

In [10]:
my_list = ['2010Hello','2010World','2011Bye','2024Welcome6191','2024Bye1700','2024FRE','2024class']
my_dict = {}

pattern = re.compile('([0-9]{4})([A-z]+)([0-9]{4})?')

for i in my_list:
    year = re.search(pattern, i)[1]
    word = re.search(pattern, i)[2]
    number = re.search(pattern, i)[3]
    if year in my_dict:
        my_dict[year][word] = number
    else:
        my_dict[year] = {word:number}
my_dict

{'2010': {'Hello': None, 'World': None},
 '2011': {'Bye': None},
 '2024': {'Welcome': '6191', 'Bye': '1700', 'FRE': None, 'class': None}}

In [11]:
# Notice that we are ignoring the .txt files
path = '.\population_growth\*.csv'

# I want to create a dictionary that keeps the years as keys and the meta/data as elements
years = {}

for file_path in glob.glob(path):
    year, meta = get_year_type(file_path)
    
    if year in years:
        years[year][meta] = fix_file(meta, pd.read_csv(file_path))
    else:
        # the structure needs to be created
        years[year] = {meta: fix_file(meta, pd.read_csv(file_path), year)}
# years

In [12]:
years.keys()

dict_keys(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'])

In [13]:
years['2010'].keys()

dict_keys(['data', 'meta'])

I want to make it so that the output of this script are 2 files: a data file and a meta file so that they can be used later

I want to then consolidate all the metas and all the data files

In [14]:
meta = pd.DataFrame()
data = pd.Series(index = pd.MultiIndex.from_tuples((), names=["year", "name"]), name = 'data')

for k, v in years.items():
    meta = pd.concat([meta, v['meta']])
    data = pd.concat([data, v['data']])

# final touches before exporting
meta = meta.drop_duplicates()
meta.index = meta.pop('id')
data = data.loc[:, meta.index]

meta.to_csv('.\data\metadata2.csv')
data.to_csv('.\data\data2.csv')

